#PEAS

This notebook demonstrates how to enhance baseline transfer attacks using the PEAS.

**Overview:**

In this notebook, you will find:
* PEAS Class Implementation: Define and utilize the PEAS class to generate adversarial examples under different augmentation scenarios.

* Parameter Definitions: Set up victim and substitute models, augmentation techniques, and attack parameters.

* Dataset Preparation: Load and preprocess the CIFAR-10 dataset, selecting correctly classified samples for evaluation.

* Baseline Transfer Attack: Define the baseline transfer attack (e.g., PGD) to be boosted by PEAS.

* Attack Success Rate (ASR) Calculation: Evaluate the effectiveness of the generated adversarial examples against a victim model.


The results will vary slightly between runs due to the random processes (augmentations and selection of images). This notebook has been setup to work on CIFAR-10 and can be adapted to other datasets. It is designed to boost any non-query based attack (e.g., BTA, PGN, TIMI).

**Getting Started:**

To begin, ensure you have all necessary libraries installed and run the cells in order. You can modify model names, parameters, and transformations as needed to fit your specific requirements. Each section includes detailed comments and explanations to guide you through the process.

##Install and Import Packages

In [3]:
import torchvision
import torch
import matplotlib.pyplot as plt
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from torchvision import transforms
from torchvision.io import read_image
from torchattacks import PGD
import numpy as np
from torchvision.transforms import v2
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import TensorDataset, DataLoader
import torchvision.datasets as datasets
from torch.utils.data.dataset import Dataset
import torch.nn as nn
import random
import pickle
import torch
from pprint import pprint
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


##PEAS Class

In [5]:
class PEAS:
    def __init__(self, F, g, S, trans, attack, count=200):
        """
        Initializes the PEAS class.

        Parameters:
        - F (list): A list of models that will be used to evaluate the generated adversarial examples.
        - g (model): The surrogate model used for generating adversarial examples.
        - S (int): The scenario for augmentation: 0 (no augmentation), 1 (single augmentation), 2 (mix of augmentations).
        - trans (list): The list of augmentations to apply.
        - count (int, optional): The number of attack iterations to perform. Defaults to 200.
        """

        self.F = F
        self.g = g
        self.S = S
        self.trans = trans
        self.count = count
        self.attack = attack

    def generate(self, x, y):

        """
        Generates adversarial examples using the selected attack under various settings.

        Parameters:
        - x (Tensor): The input images.
        - y (Tensor): The correct labels for the input images.

        Returns:
        - adv_x (Tensor): The generated adversarial examples.
        """

        batch_size = len(x)
        best_attacks = []
        min_confidences = [float('inf')] * batch_size

        for _ in tqdm(range(self.count)):


            if self.S==0: #without augmentations
                x_a = self.attack(x, y)

            if self.S==1: #PEAS S1 - one augmentation
                x_a = random.choice(self.trans)(x)
            
                images_min = clip_by_tensor(x - epsilon, 0.0, 1.0)
                images_max = clip_by_tensor(x + epsilon, 0.0, 1.0)
                x_p = PGN(x_a, y, g, images_min, images_max)

            elif self.S==2: #PEAS S2 - mixing augmentations
                augmentations = transforms.Compose(self.trans)
                x_a = augmentations(x)
                x_a = self.attack(x_a, y)

            average_confidences =  np.zeros(batch_size, dtype=np.float32)


            for model in self.F:
                outputs = model(x_a).softmax(dim=1)
                confidences = outputs[torch.arange(outputs.size(0)),y].detach().cpu().numpy()
                average_confidences += confidences
                torch.cuda.empty_cache()


            average_confidences /= len(F)

            # Compare and select the attack with the lowest average confidence for each image
            for i in range(batch_size):
                if average_confidences[i] < min_confidences[i]:
                    if len(best_attacks) < batch_size:
                        best_attacks.append(x_a[i])
                    else:
                        best_attacks[i] = x_a[i]
                    min_confidences[i] = average_confidences[i]

        # Stack the selected attacks
        adv_x = torch.stack(best_attacks)
        return adv_x

In [9]:
class model_with_normalization(nn.Module):
    def __init__(self, model, normalization):
        super(model_with_normalization, self).__init__()
        self.model = model 
        self.normalization = normalization
        
    def forward(self, x):
        x = self.normalization(x)
        if x.shape == (3,224,224):
          x = x.unsqueeze(0)
        out = self.model(x)
        return out
    
normalization = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])

##Parameters
Define the names of the victim and substitute models, and the set of augmentation algorithms for subtle transformations. You can change these parameters.

In [5]:
#The basline tranfer attack to boost by PEAS
epsilon = 2/255
#Set of augmentation algorithms, each configured to perform a subtle augmentation. S1: random augmentation from the set , S2: applies all augmentations from the set.
trans = [
    transforms.RandomAffine(degrees=(-4,4), translate=(0.1, 0.1)), # Reduced degree of rotation and translation
    transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05), # Subtle color adjustments
    transforms.RandomCrop(size=(32,32), padding=3), # Adjust padding if needed
    transforms.GaussianBlur(kernel_size=1.9), # Smaller kernel for subtle blur
    transforms.RandomAdjustSharpness(sharpness_factor=1.5), # Gentle sharpness adjustment
    transforms.RandomAutocontrast()] # Generally subtle, adjust if necessary

#Number of augmentations per input
count = 200

In [10]:
for i , (name_v, model_v, weights_v) in enumerate(models): 
    if i==0:
        victim = model_v(weights=weights_v.DEFAULT)
        print("______________________")
        print("Victim: "+ name_v)
    
        S = models.copy()
        S.pop(i)
        victim = model_with_normalization(victim, normalization)
        victim.eval()
        victim = victim.to(device)
        
        with open('../data_'+name_v+'1000images.pkl', 'rb') as f:
            new_data_loader = pickle.load(f)

        for j , (name_g,model_g, weights_g) in enumerate(S):
           print("g: "+ name_g) 
           g = model_g(weights=weights_g.DEFAULT) 
           Se = S.copy()
           Se.pop(j)
           g = model_with_normalization(g, normalization)

           g.eval()
           g = g.to(device)
           PGD_attack = PGD(g, eps=epsilon , alpha=random.uniform(0.1/255, 0.3/255), steps=random.randint(10, 20), random_start=True)
           
           
            
           F = []
           for n , model_s, weights_s in Se:
                model = model_s(weights=weights_s.DEFAULT)
                model = model.to(device)
                model = model_with_normalization(model, normalization)
                model.to(device)
                model.eval()
                F.append(model)
               
           ASR = {'Original': 0 , 'PEAS_S1': 0, 'PEAS_S2': 0}    
           for batch, (x, y) in enumerate(new_data_loader):
               
               images_min = clip_by_tensor(images - epsilon, 0.0, 1.0)
               images_max = clip_by_tensor(images + epsilon, 0.0, 1.0)
               x_p = PGN(images, labels, g, images_min, images_max)
               adv_label_p = victim(x_p).argmax(dim=1)
               adv_label = adv_label.to('cpu')
               ASR['Original'] += int((y != adv_label).int().sum())

               #S1: ONE augmentations
               PEAS_attack = PEAS(F,g, 1, trans, PGD_attack, count=count)
               x_a = PEAS_attack.generate(x, y)
               x_a = x_a.to(device)
               adv_label = victim(x_a).argmax(dim=1)
               adv_label = adv_label.to('cpu')
               ASR['PEAS_S1'] += int((y != adv_label).int().sum())
            
            
               #S2: mixing augmentations
                PEAS_attack = PEAS(F,g, 2, trans, PGD_attack, count=count)
                x_a = PEAS_attack.generate(x, y)
                x_a = x_a.to(device)
                adv_label = victim(x_a).argmax(dim=1)
                adv_label = adv_label.to('cpu')
                ASR['PEAS_S2'] += int((y != adv_label).int().sum())


print(ASR) 

______________________
Victim: vit_b_16
g: efficientnet_b0
g: swin_s
g: densenet121
g: resnet18


##Load and Normalize the Selected Models
 Load the sselected models from PyTorch Hub, apply normalization, and set the models to evaluation mode.

In [7]:
#Victim Model
victim = models[victim_name][0](weights=models[victim_name][1].DEFAULT)
victim = model_with_normalization(victim, normalization)
victim = victim.to(device)
victim.eval()


#Substitute model to generate adversarial examples
g = models[substitute_name][0](weights=models[substitute_name][1].DEFAULT)
g = model_with_normalization(g, normalization)
g.eval()
g = g.to(device)


#Set of substitute models to compute ET score
F = []
for model_name in substitute_models_name:
    model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_"+model_name, pretrained=True, verbose=False)
    model = model.to(device)
    model = model_with_normalization(model, normalization)
    model.eval()
    F.append(model)

Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/releases/download/resnet/cifar10_resnet20-4118986f.pt" to /root/.cache/torch/hub/checkpoints/cifar10_resnet20-4118986f.pt
100%|██████████| 1.09M/1.09M [00:00<00:00, 48.2MB/s]
Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/releases/download/vgg/cifar10_vgg11_bn-eaeebf42.pt" to /root/.cache/torch/hub/checkpoints/cifar10_vgg11_bn-eaeebf42.pt
100%|██████████| 37.3M/37.3M [00:00<00:00, 49.3MB/s]
Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/releases/download/mobilenetv2/cifar10_mobilenetv2_x0_5-ca14ced9.pt" to /root/.cache/torch/hub/checkpoints/cifar10_mobilenetv2_x0_5-ca14ced9.pt
100%|██████████| 2.85M/2.85M [00:02<00:00, 1.22MB/s]
Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/releases/download/shufflenetv2/cifar10_shufflenetv2_x1_5-296694dd.pt" to /root/.cache/torch/hub/checkpoints/cifar10_shufflenetv2_x1_5-296694dd.pt
100%|██████████| 9.69M/9.69M [00:00<00:00, 55.9MB/s]
Do

##Basline Tranfer Attack
Select the baseline attack to be boosted by PEAS.

##Calculate Attack Success Rate (ASR)
Calculate the ASR for three scenarios: Vanilla (no augmentations), PEAS_S1 (single augmentation), and PEAS_S2 (mix of augmentations). Generate adversarial examples using the PEAS framework and evaluate them against the victim model.

In [10]:
ASR = {'Vanilla': 0 , 'PEAS_S1': 0, 'PEAS_S2': 0}


with open('../data_'+name_v+'1000images.pkl', 'rb') as f:
        new_data_loader = pickle.load(f)
    
for batch, (x, y) in enumerate(new_data_loader):

    #vanilla-no augmetations
    PEAS_attack = PEAS(F,g, 0, trans, PGD_attack, count=count)
    x_a = PEAS_attack.generate(x, y)
    x_a = x_a.to(device)
    adv_label = victim(x_a).argmax(dim=1)
    adv_label = adv_label.to('cpu')
    ASR['Vanilla'] += int((y != adv_label).int().sum())

    #S1: ONE augmentations
    PEAS_attack = PEAS(F,g, 1, trans, PGD_attack, count=count)
    x_a = PEAS_attack.generate(x, y)
    x_a = x_a.to(device)
    adv_label = victim(x_a).argmax(dim=1)
    adv_label = adv_label.to('cpu')
    ASR['PEAS_S1'] += int((y != adv_label).int().sum())


    #S2: mixing augmentations
    PEAS_attack = PEAS(F,g, 2, trans, PGD_attack, count=count)
    x_a = PEAS_attack.generate(x, y)
    x_a = x_a.to(device)
    adv_label = victim(x_a).argmax(dim=1)
    adv_label = adv_label.to('cpu')
    ASR['PEAS_S2'] += int((y != adv_label).int().sum())


print(ASR)

100%|██████████| 200/200 [01:29<00:00,  2.22it/s]

{'Vanilla': 200, 'PEAS_S1': 363, 'PEAS_S2': 528}


In [11]:
print(f"Adversarial Success Rates (ASR) on CIFAR10, victim: {victim_name}, Substitute: {substitute_name}")
for method, rate in ASR.items():
    print(f"{method}: {rate/1000}")

Adversarial Success Rates (ASR) on CIFAR10, victim: resnet20, Substitute: vgg11_bn
Vanilla: 0.2
PEAS_S1: 0.363
PEAS_S2: 0.528
